In [1]:
%load_ext autoreload
%autoreload 1

import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

%aimport ihop.community2vec
%aimport ihop.import_data

2022-01-07 19:25:29,281 : INFO : NumExpr defaulting to 4 threads.


In [2]:
spark = ihop.utils.get_spark_session("Community2vec Notebook")

22/01/07 19:25:36 WARN Utils: Your hostname, Kurt resolves to a loopback address: 127.0.1.1; using 192.168.0.11 instead (on interface wlp4s0)
22/01/07 19:25:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/07 19:25:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/07 19:25:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/01/07 19:25:42 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


Spark configuration:
[('spark.app.startTime', '1641601538182'), ('spark.executor.id', 'driver'), ('spark.driver.host', '192.168.0.11'), ('spark.sql.warehouse.dir', 'file:/home/virginia/Documents/CenterForDataScience/ZuckermanProj/IHOP/notebooks/spark-warehouse'), ('spark.driver.port', '33613'), ('spark.app.name', 'Community2vec Notebook'), ('spark.rdd.compress', 'True'), ('spark.app.id', 'local-1641601542937'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.executor.memory', '4G'), ('spark.executor.extraLibraryPath', '/home/virginia/hadoop-3.3.1/lib/native'), ('spark.ui.showConsoleProgress', 'true'), ('spark.driver.extraLibraryPath', '/home/virginia/hadoop-3.3.1/lib/native')]


In [3]:
vocab = ihop.community2vec.get_vocabulary("../data/community2vec/sample_data/subreddit_counts.csv")
print("VOCAB SIZE:", len(vocab))
print("VOCAB SNIPPET:")
for i, (k,v) in enumerate(vocab.items()):
    print(k,":",v)
    if i>10:
        break

print("TOTAL COMMENT COUNT:", sum(vocab.values()))

VOCAB SIZE: 2275
VOCAB SNIPPET:
AskReddit : 486
CFB : 403
CrazyIdeas : 261
news : 158
ConciseIAmA : 147
4chan4trump : 136
politics : 117
RocketLeagueExchange : 96
The_Donald : 90
nba : 90
edc_raffle : 77
worldnews : 74
TOTAL COMMENT COUNT: 10000


In [6]:
from ihop.community2vec import GensimCommunity2Vec

myc2v = GensimCommunity2Vec.init_with_spark(spark, vocab, "../data/community2vec/sample_data/user_contexts/part-00000-0ad8096e-a7b4-4941-8946-0a8c7cb8108d-c000.csv.bz2")


2022-01-07 19:26:46,098 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=150, alpha=0.025)', 'datetime': '2022-01-07T19:26:46.098600', 'gensim': '4.1.2', 'python': '3.8.12 (default, Oct 12 2021, 13:49:34) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.0-92-generic-x86_64-with-glibc2.17', 'event': 'created'}
2022-01-07 19:26:46,100 : INFO : Processing provided word frequencies
2022-01-07 19:26:46,101 : INFO : collected 2275 unique word types, with total frequency of 10000
2022-01-07 19:26:46,102 : INFO : Creating a fresh vocabulary
2022-01-07 19:26:46,138 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=0 retains 2275 unique words (100.0%% of original 2275, drops 0)', 'datetime': '2022-01-07T19:26:46.138236', 'gensim': '4.1.2', 'python': '3.8.12 (default, Oct 12 2021, 13:49:34) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.0-92-generic-x86_64-with-glibc2.17', 'event': 'prepare_vocab'}
2022-01-07 19:26:46,147 : INFO : Word2Vec lifecycle event {'msg': 'effective_

In [7]:
print("CHECK INITIALIZATION")
print("Longest sentence:", myc2v.max_comments)
print("Num sentences:", myc2v.num_users)
print("Random embeddings nearest neighbor:", myc2v.w2v_model.wv.most_similar("AskReddit"))

CHECK INITIALIZATION
Longest sentence: 258
Num sentences: 7231
Random embeddings nearest neighbor: [('ImagesOfThe2010s', 0.26547014713287354), ('confession', 0.2560444176197052), ('circlejerk', 0.25592470169067383), ('FFXV', 0.23708847165107727), ('ImagesOfNetherlands', 0.22937066853046417), ('BannedFromThe_Donald', 0.22841127216815948), ('GundamExVs', 0.22638258337974548), ('neovim', 0.2199598252773285), ('Drugs', 0.21625253558158875), ('Anxiety', 0.21418383717536926)]


In [8]:
myc2v.train(compute_loss=True)

2022-01-07 19:26:59,881 : INFO : files read into PathLineSentences:../data/community2vec/sample_data/user_contexts/part-00000-0ad8096e-a7b4-4941-8946-0a8c7cb8108d-c000.csv.bz2
2022-01-07 19:26:59,882 : INFO : Word2Vec lifecycle event {'msg': 'training model with 3 workers on 2275 vocabulary and 150 features, using sg=1 hs=0 sample=0 negative=20 window=258 shrink_windows=True', 'datetime': '2022-01-07T19:26:59.882615', 'gensim': '4.1.2', 'python': '3.8.12 (default, Oct 12 2021, 13:49:34) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.0-92-generic-x86_64-with-glibc2.17', 'event': 'train'}
2022-01-07 19:26:59,935 : INFO : reading file ../data/community2vec/sample_data/user_contexts/part-00000-0ad8096e-a7b4-4941-8946-0a8c7cb8108d-c000.csv.bz2
2022-01-07 19:27:00,083 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-01-07 19:27:00,086 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-01-07 19:27:01,133 : INFO : EPOCH 1 - PROGRESS: at 100.00% examples, 769

(46105, 46105)

In [ ]:
myc2v.w2v_model.get_latest_training_loss()

218261.59375

In [9]:
myc2v.w2v_model.wv.most_similar("AskReddit", topn=10)

[('PUBATTLEGROUNDS', 0.9995643496513367),
 ('relationships', 0.9995587468147278),
 ('cars', 0.9995018839836121),
 ('RecruitCS', 0.9994935989379883),
 ('RandomActsOfBlowJob', 0.9994513988494873),
 ('sneakermarket', 0.9994426965713501),
 ('twinpeaks', 0.9993971586227417),
 ('NHLStreams', 0.9993908405303955),
 ('somethingimade', 0.9993491768836975),
 ('soccer', 0.9993222951889038)]

In [10]:
myc2v.w2v_model.wv.most_similar("news", topn=10)

[('democrats', 0.8608246445655823),
 ('science', 0.8501731753349304),
 ('vegan', 0.8471425175666809),
 ('nottheonion', 0.8452572822570801),
 ('worldnews', 0.8441250324249268),
 ('GhostAdventures', 0.8412648439407349),
 ('CAguns', 0.8408916592597961),
 ('HelloInternet', 0.8403008580207825),
 ('SULeaks', 0.8400110006332397),
 ('electricians', 0.8399454355239868)]